### Tableau Server Cliend で　サーバーから必要情報を取得

### 必要モジュールのインポート

In [2]:
import pandas as pd
import tableauserverclient as TSC
from itertools import zip_longest

In [2]:
pip list

Note: you may need to restart the kernel to use updated packages.


C:\JupyterLab\resources\jlab_server\python.exe: No module named pip


In [98]:
import sys

In [99]:
print(sys.version)

3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:22:46) [MSC v.1916 64 bit (AMD64)]


In [107]:
%pip install requests --upgrade

Note: you may need to restart the kernel to use updated packages.


C:\JupyterLab\resources\jlab_server\python.exe: No module named pip


In [94]:
%pip install -U setuptools

^C
Note: you may need to restart the kernel to use updated packages.


In [97]:
python --version

NameError: name 'python' is not defined

### ①品管所有のワークブック・ビュー・データソースを取得

In [3]:
#接続情報（ユーザーID・パスワード・URLの入力）
tableau_auth = TSC.TableauAuth('hinkan_tableau_creator', 'creator', '')
server = TSC.Server('http://10.60.28.26/')
#タブローサーバーに合わせAPIバージョンを設定
server.use_server_version()

#タブローサーバーを抽出し、
with server.auth.sign_in(tableau_auth):
    #取得ページ数　デフォルトは100件の制限を1000件へ変更(最大1000件）
    request_options = TSC.RequestOptions(pagesize=1000)
    
    all_workbook_id = []
    workbook_list =[]
    
    #空のリストへ品管所有のワークブックIDを挿入
    for wb in TSC.Pager(server.workbooks,request_options):
        if wb.owner_id.startswith('d2cdd4e4-db7f-4ccc-9cc4-adef249c9ea2'):
            all_workbook_id.append(wb.id)    
    # all_workbooks_id =([workbooks.id for workbooks in all_workbooks  ])

    #　ワークブック名・ワークブックサイズ・ビュー名・ビューID・ビュー閲覧回数・データソース名を取得
    for i in (all_workbook_id):
        workbook = server.workbooks.get_by_id(i)
        server.workbooks.populate_connections(workbook)
        server.workbooks.populate_views(workbook,usage=True)
                
        workbook_list += list(zip_longest([workbook.name]
                                ,[workbook.size]
                                ,[workbook.updated_at]
                                ,[view.name for view in workbook.views]
                                ,[view.id for view in workbook.views]
                                ,[view.total_views for view in workbook.views]
                                ,[connection.datasource_name for connection in workbook.connections]
                          ))

### DFを作成

In [4]:
workbook_df = pd.DataFrame(workbook_list,columns=['Workbook Name','Workbook Size','update_date','View Name','View ID','Total Count View','Datasource'])

### ②抽出ブック名・抽出方法・抽出スケジュールを取得

In [5]:
with server.auth.sign_in(tableau_auth):
    
    # 全てのタスクを取得
    all_tasks, pagination_item = server.tasks.get()
    
    # 全てのスケジュールを取得
    all_schedules, pagination_item = server.schedules.get()
    
    # 全てのジョブを取得
    all_jobs, pagination_item = server.jobs.get()
    
    jobs_list = [job for job in all_jobs]
    
    #print("\nThere are {} tasks on site: ".format(pagination_item.total_available))
    
    # 各taskのIDを取得してリスト化
    task_id_list = [task.id for task in all_tasks]
    
    # 各タスクのタスクタイプを取得(完全更新or増分更新)
    task_type_list = [task.task_type for task in all_tasks]
    
    # 各taksのschedule_idを取得
    task_schedule_id_list = [task.schedule_id for task in all_tasks]
    
    # task objectのtarget property を使って各タスクに紐づけられたワークブックないしデータソースを取得
    target_list = [task.target for task in all_tasks]
    
    # 各schedule objectをリスト化
    # schedule object の __repr__()は id, name propertyを使っている
    schedule_list = [schedule for schedule in all_schedules]
    
    # targetから取得したworkbook IDを使ってworkbook objectをリスト化
    workbook_object_list = []
    workbook_name_list = []
    
    for target in target_list:
        
        # target.id = workbook.id なので、それを使ってworkbook objectを取得
        work_book = server.workbooks.get_by_id(target.id)
        
        # workbook objectのリストを作成
        workbook_object_list.append(work_book)
        
        # workbook objectのname propertyを取得してリスト化
        workbook_name_list.append(work_book.name)

### task object と workbook objectを紐づけてDFを作成

In [84]:
task_df = pd.DataFrame(list(zip(workbook_name_list, task_id_list, task_type_list, task_schedule_id_list)),
               columns =['Workbook Name', 'Task ID', 'Task Type', 'Schedule ID'])

### schedule ojbect から ID と スケジュールを取得してDFを作成

In [85]:
schedule_id_list = [schedule.id for schedule in schedule_list]
schedule_name_list = [schedule.name for schedule in schedule_list]

In [86]:
schedule_df = pd.DataFrame(list(zip(schedule_id_list, schedule_name_list)),
               columns =['Schedule ID', 'Schedule'])

In [87]:
task_df =task_df.merge(schedule_df, on='Schedule ID', how='left')

### ①と②のdfを結合

In [88]:
total = workbook_df.merge(task_df, on='Workbook Name', how='left')

In [89]:
total

,Workbook Name,Workbook Size,update_date,View Name,View ID,Total Count View,Datasource,Task ID,Task Type,Schedule ID,Schedule
0,週次監視_CIW,1.0,2021-06-02 00:48:10+00:00,パレート図,9b4a550c-83c3-48a8-99b8-48ecb62142e0,68.0,CIW,NaN,NaN,NaN,NaN
1,None,NaN,NaT,時系列,cfc60f2f-aa17-4a58-8f3b-88e2221e7888,46.0,None,NaN,NaN,NaN,NaN
2,None,NaN,NaT,挿入力,012f7e7e-2198-4433-b075-1d8acda876ce,32.0,None,NaN,NaN,NaN,NaN
3,None,NaN,NaT,封口体高さ,7bd95a2e-f9a4-4c99-a076-e2222a633bd5,23.0,None,NaN,NaN,NaN,NaN
4,週次監視_CCW,1.0,2021-06-02 00:48:10+00:00,パレート図,b8100e84-34c5-4fa1-b778-4a1cb8710b71,131.0,CCW,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2256,None,NaN,NaT,正極塗布寸法(定期収集)_登録日単位,05b209a4-2e97-4e19-a69e-82b5d911ebdf,0.0,None,NaN,NaN,NaN,NaN
2257,極板パラメータ監視_4期正極塗布寸法(定期収集),1.0,2022-04-25 04:33:08+00:00,正極塗布寸法（定期収集）,01954431-fb46-476d-8663-be493d80d422,9.0,PSK3_VA370_KB_NCT_T5 (TABCREATOR),NaN,NaN,NaN,NaN
2258,None,NaN,NaT,正極塗布寸法(定期収集)_登録日単位,32a46db7-5954-48c9-9ddf-568fab7cebd6,0.0,None,NaN,NaN,NaN,NaN
2259,極板パラメータ監視_負極塗布寸法(定期収集),1.0,2022-04-25 05:08:18+00:00,負極塗布寸法(定期収集),c40f81ab-500e-4955-9536-c5109e1f4ce6,9.0,PSK3_VA370_KB_NCT_T5 (TABCREATOR),NaN,NaN,NaN,NaN


### タブローサーバーから過去90日分の閲覧数の情報を取得

In [14]:
#tableauサーバーへの接続の為、tableauserverclientをインストール
# %pip install pyautogui --proxy http://proxy.mei.co.jp:8080

Note: you may need to restart the kernel to use updated packages.


C:\JupyterLab\resources\jlab_server\python.exe: No module named pip


In [89]:
# %pip install PyMsgBox==1.0.7 --proxy http://proxy.mei.co.jp:8080

  Using cached PyMsgBox-1.0.7.tar.gz (18 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for PyMsgBox: filename=PyMsgBox-1.0.7-py3-none-any.whl size=7322 sha256=8dd83fb989b3a782bbe78af1e3470f18357c04cd4cb660cd53c9d97562db1fe0
  Stored in directory: c:\users\4076509\appdata\local\pip\cache\wheels\b3\c6\4c\98d43fb464004c5158fab6eb0cebaf63e7f9a96ff8106dc182
Successfully built PyMsgBox
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome import service as fs
import time
from time import sleep
import pyautogui
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions    import NoSuchWindowException
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
# chrome driverのpath（各自でパスの設定が必要）
# CHROMEDRIVER = "C:\\Users\\E327469\\Downloads\\chromedriver_win32\\chromedriver.exe"

CHROMEDRIVER  = "C:\\Users\\4076509\\Desktop\\chromedriver_win32\\chromedriver.exe"


In [3]:
# ドライバー指定でChromeブラウザを開く 
chrome_service = fs.Service(executable_path=CHROMEDRIVER) 
browser = webdriver.Chrome(service=chrome_service)
browser.get("http://10.60.28.26/#/analysis/TraffictoViews")
sleep(15)

In [4]:
#ログインIDを入力
browser.find_element(by='name', value="username").send_keys("hinkan_tableau_creator")

In [5]:
#パスワードを入力
browser.find_element(by='name', value="password").send_keys("creator")

In [6]:
#サインインボタンをクリック
browser.find_element(by='class name', value="login-page_submit-button_f1pvbd5k").click()
sleep(15)

In [7]:
iframe = browser.find_element(by='tag name', value='iframe')

In [8]:
#browser.switch_to.frame(iframe)
browser.switch_to.frame(iframe)

In [13]:
#時間範囲のドロップダウンボタンをクリック
browser.find_element(by='id', value="dijit_form_DropDownButton_0_label").click()

In [14]:
#過去〇〇日間のテキストボックスに90と入力
browser.find_element(by='class name', value="rtext").clear()
browser.find_element(by='class name', value="rtext").send_keys(90)
sleep(10)

In [17]:
#データボタンをクリック
browser.find_element_by_xpath('//button[normalize-space()="データ"]').click()

C:\Users\4076509\AppData\Local\Temp\ipykernel_21428\2081080853.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  browser.find_element_by_xpath('//button[normalize-space()="データ"]').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[normalize-space()="データ"]"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AE7413+2389011]
	Ordinal0 [0x00A79F61+1941345]
	Ordinal0 [0x0096C658+837208]
	Ordinal0 [0x009991DD+1020381]
	Ordinal0 [0x0099949B+1021083]
	Ordinal0 [0x009C6032+1204274]
	Ordinal0 [0x009B4194+1130900]
	Ordinal0 [0x009C4302+1196802]
	Ordinal0 [0x009B3F66+1130342]
	Ordinal0 [0x0098E546+976198]
	Ordinal0 [0x0098F456+980054]
	GetHandleVerifier [0x00C99632+1727522]
	GetHandleVerifier [0x00D4BA4D+2457661]
	GetHandleVerifier [0x00B7EB81+569713]
	GetHandleVerifier [0x00B7DD76+566118]
	Ordinal0 [0x00A80B2B+1968939]
	Ordinal0 [0x00A85988+1989000]
	Ordinal0 [0x00A85A75+1989237]
	Ordinal0 [0x00A8ECB1+2026673]
	BaseThreadInitThunk [0x775DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77D27A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77D27A4E+238]


In [27]:
#別ウィンドウへの切り替え
handle_array = browser.window_handles
browser.switch_to.window(handle_array[1])
sleep(5)

In [56]:
#csvファイルをダウンロード　Pythonのexeファイルと同じフォルダに置く
csvlink = browser.find_element(by='class name', value="csvLink_summary")
webdriver.ActionChains(browser).click(csvlink).perform()

### Tableau server client　と　Tableauサーバーから取得したデータを結合し、集計

In [90]:
#Tableauサーバーからダウンロードしたcsvファイルをdf化
viewcount_df = pd.read_csv('View_Count_per_View_data.csv')

In [91]:
#df結合の為、2つのdf間の列名を合わせる
viewcount_df.rename(columns={'名前 (履歴ビュー)':'View Name', '名前 (履歴ワークブック)':'Workbook Name'},inplace=True)
viewcount_df.columns

Index(['ビュー ID', 'View Name', 'name (hist_projects)',
       'url_namespace (site_restricted_sites)', 'View_url', 'ワークブック ID',
       'Workbook Name', '既定のサイト', 'レコード数'],
      dtype='object')

In [92]:
#2つのテーブルを左結合する
total = total.merge(viewcount_df, on=['Workbook Name','View Name'], how='left')

In [94]:
total

,Workbook Name,Workbook Size,update_date,View Name,View ID,Total Count View,Datasource,Task ID,Task Type,Schedule ID,Schedule,ビュー ID,name (hist_projects),url_namespace (site_restricted_sites),View_url,ワークブック ID,既定のサイト,レコード数
0,週次監視_CIW,1.0,2021-06-02 00:48:10+00:00,パレート図,9b4a550c-83c3-48a8-99b8-48ecb62142e0,68.0,CIW,NaN,NaN,NaN,NaN,292.0,個別分析（AF依頼分）,NaN,_CIW/sheet0,23.0,NaN,4.0
1,None,NaN,NaT,時系列,cfc60f2f-aa17-4a58-8f3b-88e2221e7888,46.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaT,挿入力,012f7e7e-2198-4433-b075-1d8acda876ce,32.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,NaN,NaT,封口体高さ,7bd95a2e-f9a4-4c99-a076-e2222a633bd5,23.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,週次監視_CCW,1.0,2021-06-02 00:48:10+00:00,パレート図,b8100e84-34c5-4fa1-b778-4a1cb8710b71,131.0,CCW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262,None,NaN,NaT,正極塗布寸法(定期収集)_登録日単位,05b209a4-2e97-4e19-a69e-82b5d911ebdf,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2263,極板パラメータ監視_4期正極塗布寸法(定期収集),1.0,2022-04-25 04:33:08+00:00,正極塗布寸法（定期収集）,01954431-fb46-476d-8663-be493d80d422,9.0,PSK3_VA370_KB_NCT_T5 (TABCREATOR),NaN,NaN,NaN,NaN,13322.0,正極3号機,NaN,_4/sheet0,1809.0,NaN,9.0
2264,None,NaN,NaT,正極塗布寸法(定期収集)_登録日単位,32a46db7-5954-48c9-9ddf-568fab7cebd6,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2265,極板パラメータ監視_負極塗布寸法(定期収集),1.0,2022-04-25 05:08:18+00:00,負極塗布寸法(定期収集),c40f81ab-500e-4955-9536-c5109e1f4ce6,9.0,PSK3_VA370_KB_NCT_T5 (TABCREATOR),NaN,NaN,NaN,NaN,13324.0,負極3号機,NaN,__16508632979940/sheet0,1810.0,NaN,9.0


###　必要データの抽出

In [95]:
#抽出を利用しているブック名の抽出
final=total[total['Task Type'].isin(['IncrementExtractTask','extractRefresh'])]

#過去90日間　閲覧が無いブックの抽出
final=final[final['レコード数'].isnull()]

In [96]:
final

,Workbook Name,Workbook Size,update_date,View Name,View ID,Total Count View,Datasource,Task ID,Task Type,Schedule ID,Schedule,ビュー ID,name (hist_projects),url_namespace (site_restricted_sites),View_url,ワークブック ID,既定のサイト,レコード数
92,外観検査 VS CCW 高さ検査,219.0,2022-04-26 23:38:53+00:00,ダッシュボード 1,0a5dc6c5-8bf3-407e-8339-a669cd204adb,43.0,VD220_KS_IV_外観検査 (PSI1USR),b041a741-7e3c-4c3b-a821-1a5a671fbddd,extractRefresh,e7144eaa-04b0-4da9-b30f-1a37acc78f19,Saturday night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,外観検査 VS CCW 高さ検査,219.0,2022-04-26 23:38:53+00:00,ダッシュボード 1,0a5dc6c5-8bf3-407e-8339-a669cd204adb,43.0,VD220_KS_IV_外観検査 (PSI1USR),9a500bcf-85a5-4f26-9415-f4c8157c1c16,IncrementExtractTask,7d594e36-b827-4935-8eb6-a6f23c4ea9fa,everyday AM8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,2号_週次監視_CCW,249.0,2022-04-23 17:23:37+00:00,正極AB_Ver3,51dd1189-bf0e-4dea-aab8-5652f522da42,101.0,CCW（tab_creator),de89033f-997a-4161-8dbf-718e1e126eee,extractRefresh,e7144eaa-04b0-4da9-b30f-1a37acc78f19,Saturday night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,2号_週次監視_CCW,249.0,2022-04-23 17:23:37+00:00,正極AB_Ver3,51dd1189-bf0e-4dea-aab8-5652f522da42,101.0,CCW（tab_creator),8703f603-acca-4d29-bb2c-7e3e2c843a24,IncrementExtractTask,c11d2e3a-7a29-4bd4-8596-353abb7f074c,everytime,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,層別不良率・計量値(封止X線),16.0,2022-04-23 17:27:50+00:00,層別不良率・計量値(CXI),0c7ed74a-875c-4746-aeee-a650f0013ca2,223.0,VC710_KM_CXI_D1_封止X線,827c71bd-2693-4b63-958e-8161ffae6a1d,IncrementExtractTask,c11d2e3a-7a29-4bd4-8596-353abb7f074c,everytime,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,#2_VI-CD,197.0,2022-04-26 23:23:00+00:00,OCVヒスト,f093465c-2c8c-453b-86dd-d5b42a4cd42d,2.0,VD130_KS_VI (PSI2USR.VD130_KS_VI) (PSI2USR),3259de51-e68d-42e7-973d-e7c957283cf6,extractRefresh,e7144eaa-04b0-4da9-b30f-1a37acc78f19,Saturday night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1282,#1_CD-OTW,82.0,2022-04-26 23:22:42+00:00,負極段差（CD時系列）,8135f6db-cc2c-475f-b6f2-745abc9f35f9,2.0,VD210_KS_OTW (PSI1USR.VD210_KS_OTW)+ (PSI1USR),8d78d7f0-8083-46ad-b00b-a54fb06f097b,extractRefresh,e7144eaa-04b0-4da9-b30f-1a37acc78f19,Saturday night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1283,#1_CD-OTW,82.0,2022-04-26 23:22:42+00:00,負極段差（CD時系列）,8135f6db-cc2c-475f-b6f2-745abc9f35f9,2.0,VD210_KS_OTW (PSI1USR.VD210_KS_OTW)+ (PSI1USR),faa7c947-4369-4e99-a077-b7256ed7289f,IncrementExtractTask,7d594e36-b827-4935-8eb6-a6f23c4ea9fa,everyday AM8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,#2_CELL-SI,77.0,2022-04-26 23:22:08+00:00,散布図,84c14a1f-c136-4230-a17a-381adea42da5,12.0,VD300_KS_SI (PSI2USR.VD300_KS_SI) (PSI2USR),4c45732d-8123-4f93-9646-06a7ac6c0afe,extractRefresh,e7144eaa-04b0-4da9-b30f-1a37acc78f19,Saturday night,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
total.to_csv("total.csv",encoding='cp932')